In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [3]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [4]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import squeezeformer, transducer

In [5]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(**config)

In [6]:
config = malaya_speech.config.squeezeformer_s_encoder_config
small_model = squeezeformer.Model(**config)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Subsampling with DS conv
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']
['M', 's', 'C', 's']


In [7]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
length_v = tf.cast(tf.shape(v)[0], tf.int32)
length_v = tf.expand_dims(length_v, 0)
v = tf.expand_dims(v, 0)
v = tf.expand_dims(v, -1)
v, length_v

(<tf.Tensor 'ExpandDims_3:0' shape=(1, ?, 80, 1) dtype=float32>,
 <tf.Tensor 'ExpandDims_1:0' shape=(1,) dtype=int32>)

In [8]:
config = malaya_speech.config.conformer_small_decoder_config
small_transducer = transducer.rnn.Model(small_model, vocabulary_size = 100, **config)

In [9]:
p = tf.placeholder(tf.int32, [None, None])
p_len = tf.placeholder(tf.int32, [None])

In [10]:
small_logits = small_transducer([v, length_v, p, p_len], training = True)
small_logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'transducer/transducer_joint/transducer_joint_vocab/BiasAdd:0' shape=(?, ?, ?, 100) dtype=float32>

In [11]:
tf.trainable_variables()

[<tf.Variable 'conformer_encoder_block_0_layer1_mhsa_scale:0' shape=(196,) dtype=float32_ref>,
 <tf.Variable 'conformer_encoder_block_0_layer1_mhsa_bias:0' shape=(196,) dtype=float32_ref>,
 <tf.Variable 'conformer_encoder_block_0_layer1_ff_scale:0' shape=(196,) dtype=float32_ref>,
 <tf.Variable 'conformer_encoder_block_0_layer1_ff_bias:0' shape=(196,) dtype=float32_ref>,
 <tf.Variable 'conformer_encoder_block_0_layer3_conv_scale:0' shape=(196,) dtype=float32_ref>,
 <tf.Variable 'conformer_encoder_block_0_layer3_conv_bias:0' shape=(196,) dtype=float32_ref>,
 <tf.Variable 'conformer_encoder_block_0_layer3_ff_scale:0' shape=(196,) dtype=float32_ref>,
 <tf.Variable 'conformer_encoder_block_0_layer3_ff_bias:0' shape=(196,) dtype=float32_ref>,
 <tf.Variable 'conformer_encoder_block_1_layer1_mhsa_scale:0' shape=(196,) dtype=float32_ref>,
 <tf.Variable 'conformer_encoder_block_1_layer1_mhsa_bias:0' shape=(196,) dtype=float32_ref>,
 <tf.Variable 'conformer_encoder_block_1_layer1_ff_scale:0' sha

In [12]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [13]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [14]:
greedy_decoded = small_transducer.greedy_decoder(v, length_v, training = False)

In [15]:
%%time

t = [[1,1,1,1,1,1]]
sess.run(small_logits, feed_dict = {i: y, p: t, p_len: [6]}).shape

CPU times: user 2.76 s, sys: 178 ms, total: 2.94 s
Wall time: 2.6 s


(1, 140, 6, 100)

In [17]:
%%time

sess.run(greedy_decoded, feed_dict = {i: y})

CPU times: user 629 ms, sys: 165 ms, total: 794 ms
Wall time: 220 ms


(array([[13, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86,
         86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86,
         86, 86, 86, 86, 86, 68, 30, 68, 68, 68, 13, 13, 13, 86, 86, 40,
         40, 40, 40, 40, 40, 40, 75, 59, 59, 13, 37, 13, 40, 40, 40, 75,
         40, 40, 40, 75, 59, 40, 40, 40, 75, 64,  3,  3,  3, 45,  3, 45,
         45,  3, 45,  3, 45,  3,  3, 90, 75,  3,  3, 45,  3, 45, 45,  3,
         45, 45,  3,  3, 45, 45,  3,  3, 45, 45, 45,  3,  3, 45, 45, 45,
          3, 45,  3,  3, 45,  3,  3, 45, 45, 45,  3, 45,  3, 45,  3, 45,
         45,  3, 45,  3, 45,  3, 45, 45,  3, 45,  3, 45]], dtype=int32),
 array([[[-6.521309 , -5.2454834, -5.192927 , ..., -6.0724506,
          -4.0811086, -4.7297845],
         [-6.7716117, -5.0538216, -5.251823 , ..., -6.004134 ,
          -4.626158 , -4.9476576],
         [-6.689    , -5.2391167, -5.4065323, ..., -5.9042387,
          -4.1656837, -5.082119 ],
         ...,
         [-6.1432815, -4.913975 , -